TP4 : Travail final


In [19]:
import os
import PyPDF2
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

In [20]:
#Lister les fichiers 
path = '../../data/camille_ecologie'

files = []
for f in os.listdir(path):
    if os.path.isfile(os.path.join(path, f)):
        files.append(f)

for f in files:
    print(f)


all.txt
BE-KBR00_15463334_19781001_99_00_00_0_01_0024_29589967.txt
BE-KBR00_15463334_19781201_100_00_00_0_01_0092_29591933.txt
BE-KBR00_15463334_19790601_101_00_00_0_01_0124_29593641.txt
BE-KBR00_15463334_19791101_103_00_00_0_01_0014_29592377.txt
BE-KBR00_15463334_19890701_137_00_00_0_01_0079_29699927.txt
KB_JB1051_1945-12-18_01-00003.txt
KB_JB1051_1951-09-05_01-00005.txt
KB_JB1051_1952-12-17_01-00004.txt
KB_JB1051_1952-12-26_01-00004.txt
KB_JB1051_1954-04-01_01-00004.txt
KB_JB1051_1958-10-15_01-00004.txt
KB_JB1051_1959-06-12_01-00001.txt
KB_JB1051_1961-03-07_01-00001.txt
KB_JB1051_1961-04-20_01-00001.txt
KB_JB1051_1961-11-20_01-00003.txt
KB_JB1051_1966-03-03_01-00002.txt
KB_JB421_1901-02-18_01-00001.txt
KB_JB421_1905-04-27_01-00001.txt
KB_JB421_1906-10-08_01-00002.txt
KB_JB421_1909-09-20_01-00001.txt
KB_JB421_1909-11-14_01-00003.txt
KB_JB421_1910-04-24_01-00002.txt
KB_JB421_1910-06-01_01-00003.txt
KB_JB427_1919-11-24_01-00002.txt
KB_JB427_1920-02-03_01-00004.txt
KB_JB427_1920-03-27_01

In [ ]:
import os

txt_path = "../../data/camille_ecologie"

with open("../../data/camille_ecologie/all.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(txt_path):
        if file.endswith(".txt") and file != "_README.txt":
            with open(os.path.join(txt_path, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())
                output_file.write("\n")   


In [27]:
all_years = [str(year) for year in range(1887, 1990)]
print(all_years)

['1887', '1888', '1889', '1890', '1891', '1892', '1893', '1894', '1895', '1896', '1897', '1898', '1899', '1900', '1901', '1902', '1903', '1904', '1905', '1906', '1907', '1908', '1909', '1910', '1911', '1912', '1913', '1914', '1915', '1916', '1917', '1918', '1919', '1920', '1921', '1922', '1923', '1924', '1925', '1926', '1927', '1928', '1929', '1930', '1931', '1932', '1933', '1934', '1935', '1936', '1937', '1938', '1939', '1940', '1941', '1942', '1943', '1944', '1945', '1946', '1947', '1948', '1949', '1950', '1951', '1952', '1953', '1954', '1955', '1956', '1957', '1958', '1959', '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986', '1987', '1988', '1989']
